#### 회귀분석
지역명, 규모, 시기, 금리, 산업생산지수, 소비자 물가지수, 부동산 정책(이슈)를 바탕으로 아파트 분양가격 예측

#### 경기 선행/후행지수인지
- 경기지표 vs 부동산 시장

#### 지역별로 그룹핑하여 PCA
-주성분 분석하여 서울지역이 지역primium이 있는지

https://datascienceschool.net/view-notebook/58269d7f52bd49879965cdc4721da42d/

In [1]:
df= pd.read_csv(
 "주택도시보증공사_전국 평균 분양가격(2019년 9월).csv", encoding='euc-kr'
)
# df["월"].unique()
df.tail()

,지역명,규모구분,연도,월,분양가격(㎡)
4075,제주,전체,2019,9,3869
4076,제주,전용면적 60㎡이하,2019,9,3804
4077,제주,전용면적 60㎡초과 85㎡이하,2019,9,3884
4078,제주,전용면적 85㎡초과 102㎡이하,2019,9,NaN
4079,제주,전용면적 102㎡초과,2019,9,3601


In [2]:
df.dtypes

지역명        object
규모구분       object
연도          int64
월           int64
분양가격(㎡)    object
dtype: object

In [3]:
#NAN값 삭제
df=df[df["분양가격(㎡)"].notnull()]

In [4]:
# "  " 삭제
df=df[df["분양가격(㎡)"]!="  "]

In [5]:
# ," 지우기
df["분양가격(㎡)"].replace(",","").tail()

4074    4303
4075    3869
4076    3804
4077    3884
4079    3601
Name: 분양가격(㎡), dtype: object

In [6]:
#분양가격 숫자형 데이터로 바꾸기
df["분양가격(㎡)"] = pd.to_numeric(df["분양가격(㎡)"], errors='coerce')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3811 entries, 0 to 4079
Data columns (total 5 columns):
지역명        3811 non-null object
규모구분       3811 non-null object
연도         3811 non-null int64
월          3811 non-null int64
분양가격(㎡)    3726 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 178.6+ KB


In [8]:
df.describe()

,연도,월,분양가격(㎡)
count,3811.000000,3811.000000,3726.000000
mean,2017.217528,6.508790,3201.605743
std,1.196525,3.449389,1223.855504
min,2015.000000,1.000000,1868.000000
25%,2016.000000,4.000000,2418.500000
50%,2017.000000,6.000000,2855.000000
75%,2018.000000,10.000000,3508.000000
max,2019.000000,12.000000,12728.000000


In [10]:
# 구글 스프레드시트에서 보기
import pygsheets
gc= pygsheets.authorize(client_secret="client_secret.json", no_cash=True)
s_file = gc.open("zigbang")
apt_sheet=s_file.add_worksheet("apt")
apt_sheet.set_dataframe(df,"A1", copy_index=False)

In [11]:
df_1=df.copy()

In [12]:
df_1["연도"]=df["연도"].apply(str)
df_1["월"]=df["월"].apply(str)
df_1["시기"]=df_1["연도"]+"0"+df_1["월"]
df_1.tail()

,지역명,규모구분,연도,월,분양가격(㎡),시기
4074,경남,전용면적 102㎡초과,2019,9,4303.0,201909
4075,제주,전체,2019,9,3869.0,201909
4076,제주,전용면적 60㎡이하,2019,9,3804.0,201909
4077,제주,전용면적 60㎡초과 85㎡이하,2019,9,3884.0,201909
4079,제주,전용면적 102㎡초과,2019,9,3601.0,201909


In [13]:
df_2=df_1.copy()
df_2.tail()

,지역명,규모구분,연도,월,분양가격(㎡),시기
4074,경남,전용면적 102㎡초과,2019,9,4303.0,201909
4075,제주,전체,2019,9,3869.0,201909
4076,제주,전용면적 60㎡이하,2019,9,3804.0,201909
4077,제주,전용면적 60㎡초과 85㎡이하,2019,9,3884.0,201909
4079,제주,전용면적 102㎡초과,2019,9,3601.0,201909


In [14]:
df_2= pd.get_dummies(df_2["지역명"])
df_2.tail()

,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
4074,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4075,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4076,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4077,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4079,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [15]:
df_3=pd.concat([df_1,df_2], axis=1)
df_3.tail()

,지역명,규모구분,연도,월,분양가격(㎡),시기,강원,경기,경남,경북,...,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
4074,경남,전용면적 102㎡초과,2019,9,4303.0,201909,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4075,제주,전체,2019,9,3869.0,201909,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4076,제주,전용면적 60㎡이하,2019,9,3804.0,201909,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4077,제주,전용면적 60㎡초과 85㎡이하,2019,9,3884.0,201909,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4079,제주,전용면적 102㎡초과,2019,9,3601.0,201909,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [16]:
#지금까지 한 df_1 csv로 저장하기
df_1.to_csv('df_1.csv',sep=",",index= False)
df_2.to_csv('df_2.csv',sep=",",index= False)
df_3.to_csv('df_3.csv',sep=",",index= False)

In [17]:
df= pd.read_csv(
 "df_1.csv")
df.tail()

,지역명,규모구분,연도,월,분양가격(㎡),시기
3806,경남,전용면적 102㎡초과,2019,9,4303.0,201909
3807,제주,전체,2019,9,3869.0,201909
3808,제주,전용면적 60㎡이하,2019,9,3804.0,201909
3809,제주,전용면적 60㎡초과 85㎡이하,2019,9,3884.0,201909
3810,제주,전용면적 102㎡초과,2019,9,3601.0,201909


In [61]:
df_3= pd.read_csv(
 "df_3.csv")
df_3.tail(11)

,지역명,규모구분,분양가격(㎡),시기,강원,경기,경남,경북,광주,대구,...,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
3800,경북,전용면적 60㎡초과 85㎡이하,2638.0,201909,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3801,경북,전용면적 85㎡초과 102㎡이하,2726.0,201909,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3802,경북,전용면적 102㎡초과,3362.0,201909,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3803,경남,전체,2995.0,201909,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3804,경남,전용면적 60㎡초과 85㎡이하,2970.0,201909,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3805,경남,전용면적 85㎡초과 102㎡이하,3211.0,201909,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3806,경남,전용면적 102㎡초과,4303.0,201909,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3807,제주,전체,3869.0,201909,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3808,제주,전용면적 60㎡이하,3804.0,201909,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3809,제주,전용면적 60㎡초과 85㎡이하,3884.0,201909,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [38]:
df_3.to_csv('df_3.csv',sep=",",index= False)

### 추가할 컬럼 가공하기

https://data.seoul.go.kr/dataList/datasetView.do?infId=524&srvType=S&serviceKind=2

In [28]:
cofix= pd.read_csv(
 "Cofix.csv", encoding='euc-kr',header=1
)
cofix_df=cofix[cofix.columns[1:3]]
cofix_df.sort_values(["대상월"], ascending=True, inplace=True)
cofix_df=cofix_df[cofix_df["대상월"]!="2014/12"]
cofix_df.reset_index(drop=True, inplace=True)
cofix_df.tail()

,대상월,신규취급액기준 COFIX
52,2019/05,1.85
53,2019/06,1.78
54,2019/07,1.68
55,2019/08,1.52
56,2019/09,1.57


In [92]:
#날짜(대상월) 형식 맞추기
cofix_df['대상월']=cofix_df['대상월'].astype("str")
cofix_df['대상월']=cofix_df['대상월'].apply(lambda x: x.replace("/",""))
cofix_df.sort_values(["대상월"], ascending=True,inplace=True)
cofix_df.columns=["date",'신규취급액기준 COFIX']
cofix_df.tail()

,date,신규취급액기준 COFIX
52,201905,1.85
53,201906,1.78
54,201907,1.68
55,201908,1.52
56,201909,1.57


In [96]:
cpi= pd.read_csv(
 "cpi.csv", encoding='euc-kr',header=0
)
cpi_df=cpi[["기간","총지수"]]
cpi_df.tail()

,기간,총지수
52,2019.05,105.31
53,2019.06,105.17
54,2019.07,104.91
55,2019.08,105.21
56,2019.09,105.57


In [97]:
#날짜(기간) 형식 맞추기
cpi_df['기간']=cpi_df['기간'].astype("str")
cpi_df['기간']=cpi_df['기간'].apply(lambda x: x.replace(".",""))
cpi_df.sort_values(["기간"], ascending=True,inplace=True)
cpi_df.columns=["date",'CPI총지수'] #칼럼명 바꾸기
cpi_df.tail()

,date,CPI총지수
52,201905,105.31
53,201906,105.17
54,201907,104.91
55,201908,105.21
56,201909,105.57


In [61]:
#산업생산지수(index of Industrial Production)
#http://www.index.go.kr/potal/main/EachDtlPageDetail.do?idx_cd=2838
iaip= pd.read_csv(
 "iaip.csv", encoding='euc-kr', header=0)
iaip=iaip.T
iaip.columns=["전산업생산지수(원지수)","전월동기대비","광공업","건설업","서비스업","공공행정"] #나중에 뭘 쓸지 몰라서 우선 모두 표시
iaip=iaip.drop(iaip.index[0])
iaip_df=pd.DataFrame(iaip["전산업생산지수(원지수)"]).reset_index() 
iaip_df.tail()

,index,전산업생산지수(원지수)
52,201905월,108.7
53,201906월,109
54,201907월,107.8
55,201908월,105.4
56,201909월,105.3


In [98]:
#날짜(index) 형식 맞추기, 칼럼명 바꾸기
iaip_df['index']=iaip_df['index'].apply(lambda x: x.replace("월",""))
iaip_df.sort_values(["index"], ascending=True,inplace=True)
iaip_df.columns=["date",'전산업생산지수(원지수)'] #칼럼명 바꾸기
iaip_df.tail()

,date,전산업생산지수(원지수)
52,201905,108.7
53,201906,109
54,201907,107.8
55,201908,105.4
56,201909,105.3


In [109]:
# 주택정책에 관한 구글 트랜드 검색값
google= pd.read_csv(
 "googletrend.csv", encoding='utf-8')
google.head()


,date,google
0,2015-01-04,29
1,2015-01-11,29
2,2015-01-18,0
3,2015-01-25,87
4,2015-02-01,59


In [110]:
google["date"]=google["date"].apply(lambda x: x[:4]+x[5:7])
google.head()

,date,google
0,201501,29
1,201501,29
2,201501,0
3,201501,87
4,201502,59


In [111]:
google_1=google.groupby("date").agg("mean").reset_index()
google_1.sort_values(["date"], ascending=True,inplace=True)
google_1.head()

,date,google
0,201501,36.25
1,201502,22.25
2,201503,29.40
3,201504,53.75
4,201505,25.40


In [112]:
#각각의 컬럼
len(cofix_df),len(cpi_df), len(iaip_df), len(google_1)

(57, 57, 57, 57)

In [113]:
#방법1 오름차순 정렬된 df들을 가로로 concat
add_df=pd.concat([cofix_df,cpi_df,iaip_df,google_1], axis=1)
add_df.tail()

,date,신규취급액기준 COFIX,date,CPI총지수,date,전산업생산지수(원지수),date,google
52,201905,1.85,201905,105.31,201905,108.7,201905,28.0
53,201906,1.78,201906,105.17,201906,109,201906,32.0
54,201907,1.68,201907,104.91,201907,107.8,201907,37.0
55,201908,1.52,201908,105.21,201908,105.4,201908,15.5
56,201909,1.57,201909,105.57,201909,105.3,201909,33.2


In [125]:
#방법2 merge (reduce함수로 누적해서 두개씩 merge)
from functools import reduce
mg_ls=[cofix_df,cpi_df,iaip_df,google_1]
add_df=reduce(lambda x,y: pd.merge(x,y),mg_ls)
add_df.tail()

,date,신규취급액기준 COFIX,CPI총지수,전산업생산지수(원지수),google
48,201905,1.85,105.31,108.7,28.0
49,201906,1.78,105.17,109,32.0
50,201907,1.68,104.91,107.8,37.0
51,201908,1.52,105.21,105.4,15.5
52,201909,1.57,105.57,105.3,33.2


In [126]:
add_df.to_csv('add_df.csv',sep=",",index= False)

In [127]:
add_df= pd.read_csv('add_df.csv')

In [123]:
#df_1불러와서 merge
df_1= pd.read_csv("df_1.csv")
df_1.columns= '지역명', '규모구분', '연도', '월', '분양가격(㎡)', 'date'
df_1.tail()

,지역명,규모구분,연도,월,분양가격(㎡),date
3806,경남,전용면적 102㎡초과,2019,9,4303.0,201909
3807,제주,전체,2019,9,3869.0,201909
3808,제주,전용면적 60㎡이하,2019,9,3804.0,201909
3809,제주,전용면적 60㎡초과 85㎡이하,2019,9,3884.0,201909
3810,제주,전용면적 102㎡초과,2019,9,3601.0,201909


In [130]:
result=pd.merge(df_1,add_df)
result.tail()

,지역명,규모구분,연도,월,분양가격(㎡),date,신규취급액기준 COFIX,CPI총지수,전산업생산지수(원지수),google
2847,경남,전용면적 102㎡초과,2019,9,4303.0,201909,1.57,105.57,105.3,33.2
2848,제주,전체,2019,9,3869.0,201909,1.57,105.57,105.3,33.2
2849,제주,전용면적 60㎡이하,2019,9,3804.0,201909,1.57,105.57,105.3,33.2
2850,제주,전용면적 60㎡초과 85㎡이하,2019,9,3884.0,201909,1.57,105.57,105.3,33.2
2851,제주,전용면적 102㎡초과,2019,9,3601.0,201909,1.57,105.57,105.3,33.2


In [136]:
#규모구분 중 전체 빼기
result_1=result[result["규모구분"]!="전체"]
result_1.tail()

,지역명,규모구분,연도,월,분양가격(㎡),date,신규취급액기준 COFIX,CPI총지수,전산업생산지수(원지수),google
2846,경남,전용면적 85㎡초과 102㎡이하,2019,9,3211.0,201909,1.57,105.57,105.3,33.2
2847,경남,전용면적 102㎡초과,2019,9,4303.0,201909,1.57,105.57,105.3,33.2
2849,제주,전용면적 60㎡이하,2019,9,3804.0,201909,1.57,105.57,105.3,33.2
2850,제주,전용면적 60㎡초과 85㎡이하,2019,9,3884.0,201909,1.57,105.57,105.3,33.2
2851,제주,전용면적 102㎡초과,2019,9,3601.0,201909,1.57,105.57,105.3,33.2
